A queue is a node in a tensorflow graph, and it's a stateful node, like a variable(other nodes can modify its content).
Queue methods (such as q.enqueue(...), q.enqueue_many(...), q.dequeue(), q.dequeue_many(...)) must run on the same device as the queue. Incompatible device placement directives will be ignored when creating these operations.
Queues such as tf.FIFOQueue and tf.RandomShuffleQueue, are important tensorflow objects for computing tensors asynchronously in a graph.

The tensorflow Session object is multithreaded, so multiple threads can easily use the same session and run ops in parallel. All threads must be able to stop together, exceptions must be caught and reported, and queues must be properly closed when stopping.
TF provides two classes to help:tf.train.Coordinator and tf.train.QueueRunner. And they're designed to be used together. The Coordinator class helps multiple threads stop together and report exceptions to a program that waits for them to stop. The QueueRunner class is used to create a number of threads cooperating to enqueue tensors in the same queue.

Coordinator class has key methods like below:
* tf.train.Coordinator.should_stop: returns True if the threads should stop.
* tf.train.Coordinator.request_stop: requests that threads should stop.
* tf.train.Coordinator.join: waits until the specified threads have stopped.
Normally, we first create a Coordinator object, and then create a number of threads that use the coordinator. The threads typically run loops that stop when should_stop return True.
Any thread can decide that the computation should stop. It only has to call request_stop() and the other threds will stop as should_stop() will then return True.

The QueueRunner class creates a number of threads that repeatedly run an enqueue op. These threads can use a coordinator to stop together. In addition, a queue runner runs a closer thread that automatically closes the queue if an exception is reported to the coordinator.
First creates a queue like: queue = tf.RandomShuffleQueue(...)
*constructor: qr = tf.train.QueueRunner(queue, [enqueue_op] * 4) #Which create a QueueRunner that will run a few, here is 4 threads to process and enqueue examples.
Then creates a coordinator and a session to launch the graph.
sess = tf.Session()
coord = tf.train.Coordinator()
*qr.create_threads(sess, coord = coord, start = True) #will lanuch the queue runner threads.

Also we can use 'threads = tf.train.start_queue_runners(coord = coord)' to start the queue runner threads.